Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [136]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [137]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [138]:
print(train_labels[0:20])

[1 5 9 4 9 1 9 9 9 4 3 0 4 2 0 2 3 9 6 3]


In [139]:
print(train_dataset[0,:,27])

[-0.49607843 -0.5        -0.42156863  0.11568628  0.4254902   0.48039216
  0.35490197  0.05686275 -0.39019608 -0.5        -0.48823529 -0.5        -0.5
 -0.5        -0.5        -0.5        -0.48823529 -0.5        -0.5        -0.5
 -0.5        -0.5        -0.48823529 -0.49607843 -0.5        -0.5        -0.5
 -0.5       ]


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [58]:
train_labels[0:2,None]

array([[1],
       [5]], dtype=int32)

In [59]:
onehot = (np.arange(10) == train_labels[0:2,None]).astype(np.int)
onehot.shape

(2, 10)

In [60]:
t = np.arange(10) == 8
t = t[None,:]
t.shape

(1, 10)

In [61]:
foo = np.array([1,2,3,4])
print(len(foo))
foo = foo.reshape(len(foo),1)
print(foo)
print(foo.shape)

4
[[1]
 [2]
 [3]
 [4]]
(4, 1)


In [62]:
#To not use GPU, on macbook, GPU memory of 512 MB is too little for most apps.
config = tf.ConfigProto(
          device_count = {'GPU': 0} )

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  #One hot mapping
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [63]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  print(tf_train_dataset.shape)
  tf_train_labels = tf.constant(train_labels[:train_subset])
  print(tf_train_labels.shape)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  print(type(weights))
  biases = tf.Variable(tf.zeros([num_labels]))
  print(type(biases))
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

(10000, 784)
(10000, 10)
<class 'tensorflow.python.ops.variables.Variable'>
<class 'tensorflow.python.ops.variables.Variable'>


Let's run this computation and iterate:

In [64]:
%%timeit -n 1 -r 1

num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph,config=config) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  #tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 20.678862
Training accuracy: 11.3%
Validation accuracy: 14.0%
Loss at step 100: 2.341020
Training accuracy: 72.0%
Validation accuracy: 70.8%
Loss at step 200: 1.882590
Training accuracy: 75.3%
Validation accuracy: 72.8%
Loss at step 300: 1.633849
Training accuracy: 76.5%
Validation accuracy: 73.7%
Loss at step 400: 1.466763
Training accuracy: 77.4%
Validation accuracy: 74.3%
Loss at step 500: 1.344389
Training accuracy: 77.9%
Validation accuracy: 74.8%
Loss at step 600: 1.249275
Training accuracy: 78.3%
Validation accuracy: 75.1%
Loss at step 700: 1.172367
Training accuracy: 78.8%
Validation accuracy: 75.3%
Loss at step 800: 1.108473
Training accuracy: 79.2%
Validation accuracy: 75.4%
Test accuracy: 82.4%
1 loop, best of 1: 24 s per loop


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [65]:

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [66]:
%%timeit -n 1 -r 1

num_steps = 3001

with tf.Session(graph=graph,config=config) as session:
  tf.global_variables_initializer().run()
  #tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.723635
Minibatch accuracy: 7.8%
Validation accuracy: 12.8%
Minibatch loss at step 500: 1.117401
Minibatch accuracy: 78.1%
Validation accuracy: 75.0%
Minibatch loss at step 1000: 1.582860
Minibatch accuracy: 72.7%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 1.302172
Minibatch accuracy: 75.8%
Validation accuracy: 77.3%
Minibatch loss at step 2000: 0.761485
Minibatch accuracy: 80.5%
Validation accuracy: 77.9%
Minibatch loss at step 2500: 0.675252
Minibatch accuracy: 79.7%
Validation accuracy: 77.9%
Minibatch loss at step 3000: 0.815709
Minibatch accuracy: 78.9%
Validation accuracy: 78.5%
Test accuracy: 85.6%
1 loop, best of 1: 3.36 s per loop


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [73]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
hidden_nodes_layer1 = 1024

def compute_logits(dataset,h_wts,h_bias,o_wts,o_bias):
    hidden_layer_linear = tf.matmul(dataset, h_wts) + h_bias
    hidden_layer_out = tf.nn.relu(hidden_layer_linear)
    logits = tf.matmul(hidden_layer_out, o_wts) + o_bias
    return logits

In [74]:
graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  print(tf_train_dataset.shape)
  tf_train_labels = tf.constant(train_labels[:train_subset])
  print(tf_train_labels.shape)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  h_weights_layer1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer1]))

  h_biases_layer1 = tf.Variable(tf.zeros([hidden_nodes_layer1]))
  
  o_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer1, num_labels]))

  o_biases = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = compute_logits(tf_train_dataset, h_weights_layer1, h_biases_layer1,
                          o_weights,o_biases)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.softmax( compute_logits(tf_valid_dataset,h_weights_layer1, h_biases_layer1,
                          o_weights,o_biases) )
    
  test_prediction = tf.nn.softmax( compute_logits(tf_test_dataset,h_weights_layer1, h_biases_layer1,
                          o_weights,o_biases) )

(10000, 784)
(10000, 10)


In [75]:
%%timeit -r 1 -n 1

num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph,config=config) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  #tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 368.148254
Training accuracy: 11.3%
Validation accuracy: 36.6%
Loss at step 100: 7.861620
Training accuracy: 89.0%
Validation accuracy: 79.0%
Loss at step 200: 5.939584
Training accuracy: 92.0%
Validation accuracy: 76.0%
Loss at step 300: 0.060244
Training accuracy: 99.5%
Validation accuracy: 79.5%
Loss at step 400: 0.155983
Training accuracy: 99.6%
Validation accuracy: 79.5%
Loss at step 500: 0.361316
Training accuracy: 99.7%
Validation accuracy: 79.5%
Loss at step 600: 0.304088
Training accuracy: 99.7%
Validation accuracy: 79.6%
Loss at step 700: 0.091009
Training accuracy: 98.7%
Validation accuracy: 79.6%
Loss at step 800: 0.234545
Training accuracy: 99.7%
Validation accuracy: 79.6%
Test accuracy: 86.8%
1 loop, best of 1: 8min 44s per loop


#### SGD on Neural Network : 1 hidden layer

In [134]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100000
hidden_nodes_layer1 = 4096

def compute_logits(dataset,h1_wts,h1_bias,o_wts,o_bias):
    h1_linear = tf.matmul(dataset, h1_wts) + h1_bias
    h1_out = tf.nn.relu(h1_linear)
    logits = tf.matmul(h1_out, o_wts) + o_bias
    return logits

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  h_weights_layer1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer1]))

  h_biases_layer1 = tf.Variable(tf.zeros([hidden_nodes_layer1]))
  
  o_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer1, num_labels]))

  o_biases = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = compute_logits(tf_train_dataset, h_weights_layer1, h_biases_layer1,
                          o_weights,o_biases)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.softmax( compute_logits(tf_valid_dataset,h_weights_layer1, h_biases_layer1,
                          o_weights,o_biases) )
    
  test_prediction = tf.nn.softmax( compute_logits(tf_test_dataset,h_weights_layer1, h_biases_layer1,
                          o_weights,o_biases) )




Run it

In [135]:
%%timeit -n 1 -r 1

num_steps = 12001

with tf.Session(graph=graph,config=config) as session:
  tf.global_variables_initializer().run()
  #tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 700.681274
Minibatch accuracy: 10.9%
Validation accuracy: 41.6%
Minibatch loss at step 500: 20.799351
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 36.575172
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 55.196560
Minibatch accuracy: 77.3%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 12.214342
Minibatch accuracy: 88.3%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 6.155316
Minibatch accuracy: 85.2%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 18.144722
Minibatch accuracy: 85.2%
Validation accuracy: 85.1%
Minibatch loss at step 3500: 10.931046
Minibatch accuracy: 87.5%
Validation accuracy: 84.5%
Minibatch loss at step 4000: 22.899525
Minibatch accuracy: 80.5%
Validation accuracy: 84.5%
Minibatch loss at step 4500: 10.038279
Minibatch accuracy: 86.7%
Validation accuracy: 83.2%
Minibatch loss at step 5000: 9.560882
Minibatch accuracy: 89.8%
Validati